*11. Write a script that stores all predicate clauses of the Hebrew part of the Hebrew Bible in a dictionary. For each clause you store the verb and the prepositions '>L' or '<D'. If there are no prepositions in a given clause, the dictionary entry should be marked as empty. For the verb, appart from the lexeme itself, you should also store information regarding stem and tense. Finally, store the dictionary in a csv file.

In [8]:
import sys, collections, os

import pprint as pp

In [9]:
from tf.fabric import Fabric
from tf.extra.bhsa import Bhsa

BHSA = f'etcbc/bhsa/tf/c'
TF = Fabric(modules=BHSA)

This is Text-Fabric 5.5.24
Api reference : https://dans-labs.github.io/text-fabric/Api/General/
Tutorial      : https://github.com/Dans-labs/text-fabric/blob/master/docs/tutorial.ipynb
Example data  : https://github.com/Dans-labs/text-fabric-data

114 features found and 0 ignored


In [10]:
#load the features that you need here! lex is given already.
    
api = TF.load('''
    lex sp gn nu vt vs typ function prs book book@en language
''', silent = True)

In [11]:
api.loadLog()
api.makeAvailableIn(globals())

   |     0.26s B otype                from C:/Users/Saturnus/text-fabric-data/etcbc/bhsa/tf/c
   |     2.16s B oslots               from C:/Users/Saturnus/text-fabric-data/etcbc/bhsa/tf/c
   |     0.02s B book                 from C:/Users/Saturnus/text-fabric-data/etcbc/bhsa/tf/c
   |     0.02s B chapter              from C:/Users/Saturnus/text-fabric-data/etcbc/bhsa/tf/c
   |     0.02s B verse                from C:/Users/Saturnus/text-fabric-data/etcbc/bhsa/tf/c
   |     0.45s B g_cons               from C:/Users/Saturnus/text-fabric-data/etcbc/bhsa/tf/c
   |     0.63s B g_cons_utf8          from C:/Users/Saturnus/text-fabric-data/etcbc/bhsa/tf/c
   |     0.45s B g_lex                from C:/Users/Saturnus/text-fabric-data/etcbc/bhsa/tf/c
   |     0.63s B g_lex_utf8           from C:/Users/Saturnus/text-fabric-data/etcbc/bhsa/tf/c
   |     0.51s B g_word               from C:/Users/Saturnus/text-fabric-data/etcbc/bhsa/tf/c
   |     0.74s B g_word_utf8          from C:/Users/Saturnus

In [12]:
import collections

In [13]:
B = Bhsa(api, 'search', version="c")

**Documentation:** <a target="_blank" href="https://etcbc.github.io/bhsa" title="{provenance of this corpus}">BHSA</a> <a target="_blank" href="https://etcbc.github.io/bhsa/features/hebrew/c/0_home.html" title="{CORPUS.upper()} feature documentation">Feature docs</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/Bhsa/" title="BHSA API documentation">BHSA API</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/General/" title="text-fabric-api">Text-Fabric API 5.5.24</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/General/#search-templates" title="Search Templates Introduction and Reference">Search Reference</a>

*11. Write a script that stores all predicate clauses of the Hebrew part of the Hebrew Bible in a dictionary. For each clause you store the verb and the prepositions '>L' or '<D'. If there are no prepositions in a given clause, the dictionary entry should be marked as empty. For the verb, appart from the lexeme itself, you should also store information regarding stem and tense. Finally, store the dictionary in a csv file.

Maybe there is some error as far as mention is made of predicate clauses but I see only predicate phrases.

### First try in Python

In [20]:
pred_clause_dict = {}
pred_clause_list = []
for clause in F.otype.s('clause'):
    phrases = L.d(clause, 'phrase')#step down to phrases
    for phrase in phrases:
        #if F.function.v(phrase) in ['Pred', 'PreC', 'PreO', 'PreS']:#find all pred clauses(?) PreC omitted
        if F.function.v(phrase) in ['Pred', 'PreO', 'PreS']:#find all pred clauses(?)    
            clause_1 = L.u(phrase, 'clause')[0]#step up to clause
            words = L.d(clause_1, 'word')#step down to words
            for word in words:#look within phrase to all words
                if F.language.v(word)  == 'Hebrew':
                    if F.sp.v(word) == 'verb':# if word = verb
                        where = T.sectionFromNode(word)
                        verb_lex = F.lex.v(word)
                        verb_tense = F.vt.v(word)
                        verb_stem = F.vs.v(word)
                        #print(verb_lex, verb_tense, verb_stem)
                    else:#if word is not a verb
                        if F.sp.v(word) == 'prep':#if it is prep
                            if F.lex.v(word) == '>L':
                                L_word = F.lex.v(word)
                                #print(T.sectionFromNode(word), L_word)
                            elif F.lex.v(word) == '<D':
                                D_word = F.lex.v(word)
                            else:
                                L_word = 'NA'
                                D_word = 'NA'

            
            #where = T.sectionFromNode(clause)
            info = [clause, where[0], where[1], where[2], verb_lex, verb_tense, verb_stem, L_word, D_word]#collect info
            pred_clause_list.append(clause)#append node nr to list
            pred_clause_dict[clause] = info#append info to dict 
            
print(len(pred_clause_list))

I make a file from the data in the dict.

In [21]:
with open(r'pred_clause.csv', "w") as csv_file:
    
    # we make a header again
    header = ['slot', 'book', 'chapter', 'verse', 'verb_lex', 'verb_tense', 'verb_stem','L_word', 'D_word']
    csv_file.write('{}\n'.format(','.join(header)))

    for case in pred_clause_list:
        info_list = pred_clause_dict[case]
        line = [str(element) for element in info_list]
        csv_file.write('{}\n'.format(','.join(line)))

### Second try with TF and Python

You will see that I insert several print commands to see what value the variables have. I always used this to keep track of what I am doing. I also tried to make an extra line "word lex#<D|>L sp=prep" to be sure that i have a possibility to mark the absence of a prepostion <D or >L. The result of this query are relevant tuples in a "frame" (maybe this is a fixed term for something else. Array would also do.) from where I can extract all the information needed. I saw no way to cretae differend places for the <L and >D preps as I did in try one.

In [ ]:
query = '''
clause
    phrase  function=Pred|PreO|PreS|PreC 
      word sp=verb
    phrase  typ=PP
      word lex=<D|>L
      
'''
results = B.search(query)
gegevens=(results)
#B.table(results, start =1, end=5)
#XB.show(results, start =1, end=5,  colorMap={2: 'cyan', 4: 'magenta'})
B.table(results, start =1, end=5)
#print(gegevens)

Below I tried to unpack the information from the tuples. I did know from the B.table the index numbers and the type of the nodes. Strange enough is that the print command hidden after # returns the correct values but I cannot attach these values to the named variables. I also don't know how to abbreviate the line 2 with  ''' so that it can flow over more lines. Putting single quotes before and after didn't do the trick. So I didn't succeed in storing the information in a file.

In [ ]:
for element in gegevens: 
    #print(element[0], T.sectionFromNode(element[0])[0],T.sectionFromNode(element[0])[1],T.sectionFromNode(element[0])[2], F.lex.v(element[2]), F.typ.v(element[3]), F.lex.v(element[4]), 'niet')
    node = str(element[0]
    book_name = T.sectionFromNode(element[0])[0]
    chapter = T.sectionFromNode(element[0])[1]
    verse = T.sectionFromNode(element[0])[2]
    verb = F.lex.v(element[2])
    phrase = F.typ.v(element[3])
    prepLD = F.lex.v(element[4])
    prepNA = F.lex.v(element[5])
    info = [node, book_name, chapter, verse, verb, phrase,prepLD,prepNA]
               
print(info[:10])

*12. Use the Python package Pandas to create a table of the csv file just created: studying this panda table I saw that there are many du- and triplicates included. Genesis 1 2 consist of three phrases but the PreC has no verb so should be omitted. But the third clause in Gen. 1 2  has a verb (partc) and is also omitted. These are time consuming things! But the file is approx. 20.000 rows shorter.

In [17]:
import pandas as pd

In [22]:
dataframe = pd.read_csv("pred_clause.csv")#pd staat voor pandas


In [23]:
dataframe

,slot,book,chapter,verse,verb_lex,verb_tense,verb_stem,L_word,D_word
0,427553,Genesis,1,1,BR>[,perf,qal,NaN,NaN
1,427554,Genesis,1,2,HJH[,perf,qal,NaN,NaN
2,427557,Genesis,1,3,>MR[,wayq,qal,NaN,NaN
3,427558,Genesis,1,3,HJH[,impf,qal,NaN,NaN
4,427559,Genesis,1,3,HJH[,wayq,qal,NaN,NaN
5,427560,Genesis,1,4,R>H[,wayq,qal,NaN,NaN
6,427561,Genesis,1,4,VWB[,perf,qal,NaN,NaN
7,427562,Genesis,1,4,BDL[,wayq,hif,NaN,NaN
8,427563,Genesis,1,5,QR>[,wayq,qal,NaN,NaN
9,427564,Genesis,1,5,QR>[,perf,qal,NaN,NaN


*13 Make a subset of the table from the preceding exercise, consisting of the 100 most frequent verbs:


This is where the buck stops!